In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from libs.calibration import CameraCalibrator
from libs.common import get_images_in_dir, plot_images, get_s_channel
from libs.filter_image import abs_sobel_thresh, mag_thresh, dir_threshold, hls_select
from libs.find_lanes import fit_polynomial

In [ ]:
# %matplotlib qt

In [ ]:
calibration_data_dir = 'camera_cal/'

calibrator = CameraCalibrator(9, 6)

calibrate_iages = get_images_in_dir(calibration_data_dir)
used_images = []

for cal_image in calibrate_iages:
    if calibrator.calibrate(cal_image):
        used_images.append(cal_image)


calibrator.finish_calibrate()

In [ ]:
def transform_image(image):
    return calibrator.process_image(image)[-1]

In [ ]:
test_images_dir = 'test_images/'
test_images = get_images_in_dir(test_images_dir)
test_images = list(map(transform_image, test_images))
train_image = test_images[2]

In [ ]:
def check_lims(image, method, min_vals, max_vals):
    ploty = np.array(range(image.shape[0])).astype(np.float64)
    fig = plt.figure(figsize=[5 * len(max_vals), 5 * len(min_vals)])
    for min_id, min_val in enumerate(min_vals):
        images = [method(image, min_val, max_val) for max_val in max_vals]
        axes = plot_images(fig, images, min_id, len(min_vals))
        for img, max_val, ax in zip(images, max_vals, axes):
            ax.set_title("({},{})".format(min_val, max_val))
            try:
                left, right = fit_polynomial(img.astype(np.uint8))
                ax.plot(left, ploty, "yellow")
                ax.plot(right, ploty, "yellow")
            except :
                pass
            

### Sobol x

In [ ]:
check_lims(
    get_s_channel(train_image),
    lambda image, min_val, max_val: abs_sobel_thresh(image, orient='x', thresh_min=min_val, thresh_max=max_val, sobel_kernel=3),
    range(4, 7, 1),
    range(100, 255, 50),)

### Sobol y

In [ ]:
check_lims(
    get_s_channel(train_image),
    lambda image, min_val, max_val: abs_sobel_thresh(image, orient='y', thresh_min=min_val, thresh_max=max_val),
    range(6, 10, 1),
    range(100, 170, 20),
)

### Magnitude of the Gradient

In [ ]:
check_lims(
    get_s_channel(train_image),
    lambda image, min_val, max_val: mag_thresh(image, sobel_kernel=3, mag_thresh=(min_val, max_val)),
    range(8, 10, 1),
    range(50, 251, 50)
)

### Direction of the Gradient

In [ ]:
check_lims(
    get_s_channel(train_image),
    lambda image, min_val, max_val: dir_threshold(image, sobel_kernel=7, thresh=(min_val, max_val)),
    [0, 0.0001, 0.001,],
    [1, 7e-1, 5e-1, 3e-1,1e-1]
)

### HLS

In [ ]:
check_lims(
    train_image,
    lambda image, min_val, max_val: hls_select(image, thresh=(min_val, max_val)),
    range(80, 150, 10),
    range(100, 255, 50)
)

## Combine

In [ ]:
sobelx = lambda img : abs_sobel_thresh(get_s_channel(img), orient='x', sobel_kernel=5, thresh_min=3, thresh_max=150)
sobely = lambda img : abs_sobel_thresh(get_s_channel(img), orient='y', sobel_kernel=5, thresh_min=7, thresh_max=150)
dirsobel = lambda img : dir_threshold(get_s_channel(img), sobel_kernel=3, thresh=(0, 0.3))
def combine_or(image):
    sybinary = sobelx(image)
    sxbinary = sobely(image)
    
    combined_binary = np.zeros_like(sybinary)
    combined_binary[(sxbinary == 1) | (sybinary == 1)] = 1
    return combined_binary.astype(np.uint8)

def combine_and(image):
    sybinary = sobelx(image)
    sxbinary = sobely(image)
    
    combined_binary = np.zeros_like(sybinary)
    combined_binary[(sxbinary == 1) & (sybinary == 1)] = 1
    return combined_binary.astype(np.uint8)

def combine(image):    
    sybinary = sobelx(image)
    sxbinary = sobely(image)
    dirbindary = dirsobel(image)
    
    combined_binary = np.zeros_like(sybinary)
    combined_binary[((sxbinary == 1) | (sybinary == 1)) & (dirbindary == 1)] = 1
    return combined_binary.astype(np.uint8)

# dir_threshold(image, sobel_kernel=7, thresh=(0, 0.5))

methods = [
#     sobelx,
#     sobely,
#     combine_or,
#     combine_and,
#     dirsobel,
    combine
]

fig = plt.figure(figsize=[5 * (len(methods) + 1), 5 * len(test_images)])
for image_id, timage in enumerate(test_images):
    images = [timage] + [method(timage) for method in methods]
    axes = plot_images(fig, images, image_id, len(test_images))
    axes[0].set_title("image {}".format(image_id))
    for img, ax in zip(images, axes):
        if len(img.shape) == 3:
            continue
        left, right, *data = fit_polynomial(img.astype(np.uint8))
        ploty = np.array(range(img.shape[0])).astype(np.float64)
        ax.plot(left, ploty, "yellow")
        ax.plot(right, ploty, "yellow")
    